## CDS Assignment

#### Dataframe Creation

In [32]:
import pandas as pd
import numpy as np
import re

In [33]:
# instantiate main dataframe, df

df = pd.DataFrame()

df['Dx ID'] = 0
df['Mx ID'] = 0
df['Dx Mx Association'] = 0
df['Dx Name'] = 0
df['Mx Name'] = 0
df['Mx PPV'] = 0
df['Mx Sensitivity'] = 0
df['Specificity'] = 0
df['LR (+)'] = 0
df['LR (-)'] = 0

df

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-)


In [34]:
# incorporate the associations between Dx and Mx
df_DxMx = pd.read_csv('dx_mx_associations.txt', sep='\t', names=['dx', 'mx'])
df_DxMx.head()

,dx,mx
0,DX 362 PEPTIC ULCER,MX 34 59 ABDOMEN PAIN EPIGASTRIUM RECURRENT AT...
1,DX 362 PEPTIC ULCER,MX 22 62 ABDOMEN PAIN EPIGASTRIUM SEASONAL HX
2,DX 362 PEPTIC ULCER,MX 23 64 ABDOMEN PAIN EXACERBATION WITH ALCOHOL
3,DX 362 PEPTIC ULCER,MX 02 198 AGE 16 TO 25
4,DX 362 PEPTIC ULCER,MX 03 199 AGE 26 TO 55


In [35]:
# read Dx Mx file

f = open('dx_mx_associations.txt')
lines = f.readlines()
f.close

lines[:5]

['DX 362  PEPTIC ULCER \tMX 34 59 ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX\n',
 'DX 362  PEPTIC ULCER \tMX 22 62 ABDOMEN PAIN EPIGASTRIUM SEASONAL HX\n',
 'DX 362  PEPTIC ULCER \tMX 23 64 ABDOMEN PAIN EXACERBATION WITH ALCOHOL\n',
 'DX 362  PEPTIC ULCER \tMX 02 198 AGE 16 TO 25\n',
 'DX 362  PEPTIC ULCER \tMX 03 199 AGE 26 TO 55\n']

In [36]:
# group the Dx ID, Dx name, Dx prevalence, Mx ID, Mx name, Mx PPV, and Mx Sensitivity

m = re.compile(r'DX\s(\d+)\s+(.+)\s\sMX\s(\d)(\d)\s(\d+)\s(.+)')

dx_id = []
dx_name = []
mx_id = []
mx_name = []
mx_ppv = []
mx_sens = []

for i in range(len(lines)):
    dx_id.append((m.match(lines[i]).group(1)))
    dx_name.append((m.match(lines[i]).group(2)))
    mx_id.append((m.match(lines[i]).group(5)))
    mx_name.append((m.match(lines[i]).group(6)))
    mx_ppv.append((m.match(lines[i]).group(3)))
    mx_sens.append((m.match(lines[i]).group(4)))

In [37]:
# combine Dx and Mx IDs for associations column

dx_mx = []

for i in range(len(dx_id)):
    dx_mx.append(dx_id[i] + ', ' + mx_id[i])
    
dx_mx[:5]

['362, 59', '362, 62', '362, 64', '362, 198', '362, 199']

In [38]:
# incorporate prevalence table
df_prev = pd.read_csv('dx_prev.txt', sep='\t')

df_prev.rename(index=str, columns={"id": "Dx ID", "prev": "Dx Prevalence", "name": "Dx Name"}, inplace=True)

df_prev['Dx ID'] = df_prev['Dx ID'].apply(str)
df_prev.head()

,Dx ID,Dx Prevalence,Dx Name
0,114,3,choledocholithiasis
1,362,5,peptic ulcer
2,441,2,pyloric obstruction
3,187,3,gastric carcinoma
4,189,2,gastritis giant hypertrophic <menetriers>


In [39]:
# incorporate import numbers
f = open('mx.txt')
lines = f.readlines()
lines[:5]

['MX 100 ABDOMEN SURGERY RECENT HX    IM  1 TY 1 \n',
 'MX 1001 COLON BARIUM ENEMA WALL THICKENED   IM  3 TY 5 \n',
 'MX 1005 COLONOSCOPY BIOPSY GRANULOMA S NON CASEATING   IM  4 TY 6 \n',
 'MX 1012 COLONOSCOPY MULTIPLE NODULAR FILLING DEFECT S   IM  4 TY 6 \n',
 'MX 102 ABDOMEN TENDERNESS EPIGASTRIUM NON HEPATIC   IM  3 TY 3 \n']

In [40]:
# group the Mx ID numbers and import numbers

m = re.compile(r'MX\s(\d+).+IM\s+(\d)')

mx_import_id = []
mx_import_num = []

for i in range(len(lines)):
    mx_import_id.append((m.match(lines[i]).group(1)))
    mx_import_num.append((m.match(lines[i]).group(2)))

In [41]:
# create dataframe for imports
df_mx_import = pd.DataFrame()
df_mx_import['Mx ID'] = mx_import_id
df_mx_import['Import Num'] = mx_import_num

In [42]:
df_mx_import.tail()

,Mx ID,Import Num
348,980,3
349,982,4
350,985,3
351,989,4
352,998,3


#### Add everything to df

In [43]:
# put associations, Dx name, Mx name, PPV, and sensitivity into df

df['Dx ID'] = dx_id
df['Mx ID'] = mx_id
df['Dx Mx Association'] = dx_mx
df['Dx Name'] = dx_name
df['Mx Name'] = mx_name
df['Mx PPV'] = mx_ppv
df['Mx Sensitivity'] = mx_sens

df.head()

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-)
0,362,59,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,3,4,NaN,NaN,NaN
1,362,62,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,2,2,NaN,NaN,NaN
2,362,64,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,2,3,NaN,NaN,NaN
3,362,198,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0,2,NaN,NaN,NaN
4,362,199,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0,3,NaN,NaN,NaN


In [44]:
df = pd.merge(df, df_mx_import, on='Mx ID')

In [45]:
df = pd.merge(df, df_prev[['Dx ID', 'Dx Prevalence']], on='Dx ID')

In [46]:
# cast all numeric values to integers 
cols = ['Dx Prevalence', 'Mx PPV', 'Mx Sensitivity', 'Import Num']
df[cols] = df[cols].apply(pd.to_numeric)
df.head()

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-),Import Num,Dx Prevalence
0,362,59,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,3,4,NaN,NaN,NaN,2,5
1,362,62,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,2,2,NaN,NaN,NaN,2,5
2,362,64,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,2,3,NaN,NaN,NaN,2,5
3,362,198,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0,2,NaN,NaN,NaN,3,5
4,362,199,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0,3,NaN,NaN,NaN,3,5


In [47]:
PREV = {1: 1/1000000,
       2: 1/100000,
       3: 1/1000,
       4: 1/100,
       5: 1/11}

PPV = {0: np.NaN,
      1: 0.01,
      2: 0.1,
      3: 0.75,
      4: 0.9,
      5: 0.9999}

SENS = {1: 0.01,
       2: 0.1,
       3: 0.5,
       4: 0.75,
       5: 0.9999}

df['Dx Prevalence'] = df['Dx Prevalence'].map(PREV)
df['Mx PPV'] = df['Mx PPV'].map(PPV)
df['Mx Sensitivity'] = df['Mx Sensitivity'].map(SENS)

In [48]:
df.head()

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-),Import Num,Dx Prevalence
0,362,59,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.75,0.75,NaN,NaN,NaN,2,0.090909
1,362,62,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.10,0.10,NaN,NaN,NaN,2,0.090909
2,362,64,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.10,0.50,NaN,NaN,NaN,2,0.090909
3,362,198,"362, 198",PEPTIC ULCER,AGE 16 TO 25,NaN,0.10,NaN,NaN,NaN,3,0.090909
4,362,199,"362, 199",PEPTIC ULCER,AGE 26 TO 55,NaN,0.50,NaN,NaN,NaN,3,0.090909


In [49]:
# add specificity helper function

def calc_specificity(sens, ppv, prev):
    """
    Calculate specificity based on sensitivity, positive
    predictive value, & prevalence. 
    """
    spec = 1 - ((sens*prev/ppv - sens*prev)/(1-prev))
    return spec

In [50]:
# calculate specificities
df['Specificity'] = calc_specificity(df['Mx Sensitivity'], df['Mx PPV'], df['Dx Prevalence'])
df.head()

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-),Import Num,Dx Prevalence
0,362,59,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.75,0.75,0.975,NaN,NaN,2,0.090909
1,362,62,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.10,0.10,0.910,NaN,NaN,2,0.090909
2,362,64,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.10,0.50,0.550,NaN,NaN,2,0.090909
3,362,198,"362, 198",PEPTIC ULCER,AGE 16 TO 25,NaN,0.10,NaN,NaN,NaN,3,0.090909
4,362,199,"362, 199",PEPTIC ULCER,AGE 26 TO 55,NaN,0.50,NaN,NaN,NaN,3,0.090909


In [51]:
def lr_pos(sens, spec):
    return sens/(1-spec)

def lr_neg(sens, spec):
    return (1-sens)/spec

df['LR (+)'] = lr_pos(df['Mx Sensitivity'], df['Specificity'])
df['LR (-)'] = lr_neg(df['Mx Sensitivity'], df['Specificity'])

In [52]:
# exploring which negative values are present (and possibly need to be corrected)
negs = df.loc[df['LR (-)'] < 0]
negs

,Dx ID,Mx ID,Dx Mx Association,Dx Name,Mx Name,Mx PPV,Mx Sensitivity,Specificity,LR (+),LR (-),Import Num,Dx Prevalence


In [53]:
# create neater version for exporting as CSV

df_final = pd.DataFrame()

df_final['Association'] = df['Dx Mx Association']
df_final['Dx Name'] = df['Dx Name']
df_final['Mx Name'] = df['Mx Name']
df_final['Prevalence'] = df['Dx Prevalence']
df_final['Import'] = df['Import Num']
df_final['LR (+)'] = df['LR (+)']
df_final['LR (-)'] = df['LR (-)']

df_final.head()

,Association,Dx Name,Mx Name,Prevalence,Import,LR (+),LR (-)
0,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.090909,2,30.000000,0.256410
1,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.090909,2,1.111111,0.989011
2,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.090909,2,1.111111,0.909091
3,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0.090909,3,NaN,NaN
4,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0.090909,3,NaN,NaN


In [54]:
all_assocs = pd.read_csv('all_assocs.csv')

df_all = pd.DataFrame()

all_assocs.head()

,dx_id,dx_name,mx_id,mx_name,associations
0,114,CHOLEDOCHOLITHIASIS,5,ABDOMEN BRUIT PRESENT,"114,5"
1,114,CHOLEDOCHOLITHIASIS,6,ABDOMEN BRUIT SYSTOLIC EPIGASTRIUM,"114,6"
2,114,CHOLEDOCHOLITHIASIS,18,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS <...,"114,18"
3,114,CHOLEDOCHOLITHIASIS,19,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS U...,"114,19"
4,114,CHOLEDOCHOLITHIASIS,32,ABDOMEN DISTENTION,"114,32"


In [55]:
df_all['Dx Name'] = all_assocs['dx_name']
df_all['Mx Name'] = all_assocs['mx_name']
df_all.head()

,Dx Name,Mx Name
0,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT PRESENT
1,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT SYSTOLIC EPIGASTRIUM
2,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS <...
3,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS U...
4,CHOLEDOCHOLITHIASIS,ABDOMEN DISTENTION


In [56]:
df_test = df_all.merge(df_final, how = 'left', on = ['Dx Name', 'Mx Name'])

In [57]:
df_test = df_test.drop(['Prevalence'], axis=1)
df_test = df_test.drop(['Association'], axis=1)

df_test.head()

,Dx Name,Mx Name,Import,LR (+),LR (-)
0,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT PRESENT,NaN,NaN,NaN
1,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT SYSTOLIC EPIGASTRIUM,NaN,NaN,NaN
2,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS <...,NaN,NaN,NaN
3,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS U...,NaN,NaN,NaN
4,CHOLEDOCHOLITHIASIS,ABDOMEN DISTENTION,NaN,NaN,NaN


In [58]:
df_prev.head()

,Dx ID,Dx Prevalence,Dx Name
0,114,3,choledocholithiasis
1,362,5,peptic ulcer
2,441,2,pyloric obstruction
3,187,3,gastric carcinoma
4,189,2,gastritis giant hypertrophic <menetriers>


In [59]:
df_prev['Dx Name'] = df_prev['Dx Name'].str.upper()

df_prev

,Dx ID,Dx Prevalence,Dx Name
0,114,3,CHOLEDOCHOLITHIASIS
1,362,5,PEPTIC ULCER
2,441,2,PYLORIC OBSTRUCTION
3,187,3,GASTRIC CARCINOMA
4,189,2,GASTRITIS GIANT HYPERTROPHIC <MENETRIERS>
5,351,4,PANCREATITIS ACUTE
6,352,3,PANCREATITIS CHRONIC
7,188,2,GASTRIC LYMPHOMA
8,192,2,GASTROINTESTINAL SARCOIDOSIS
9,616,1,ANISAKIASIS


In [60]:
df_test = pd.merge(df_test, df_prev[['Dx Name', 'Dx Prevalence']], how='left', on='Dx Name')

df_test.head()

,Dx Name,Mx Name,Import,LR (+),LR (-),Dx Prevalence
0,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT PRESENT,NaN,NaN,NaN,3
1,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT SYSTOLIC EPIGASTRIUM,NaN,NaN,NaN,3
2,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS <...,NaN,NaN,NaN,3
3,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS U...,NaN,NaN,NaN,3
4,CHOLEDOCHOLITHIASIS,ABDOMEN DISTENTION,NaN,NaN,NaN,3


In [61]:
df_test['LR (+)'] = df_test['LR (+)'].fillna(value = 1)
df_test['LR (-)'] = df_test['LR (-)'].fillna(value = 1)

# convert prevalances back to proportions
df_test['Prevalence'] = df_test['Dx Prevalence'].map(PREV)

df_test

,Dx Name,Mx Name,Import,LR (+),LR (-),Dx Prevalence,Prevalence
0,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT PRESENT,NaN,1.000000,1.000000,3,0.001000
1,CHOLEDOCHOLITHIASIS,ABDOMEN BRUIT SYSTOLIC EPIGASTRIUM,NaN,1.000000,1.000000,3,0.001000
2,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS <...,NaN,1.000000,1.000000,3,0.001000
3,CHOLEDOCHOLITHIASIS,ABDOMEN COMPUTERIZED TOMOGRAPHY ADRENAL MASS U...,NaN,1.000000,1.000000,3,0.001000
4,CHOLEDOCHOLITHIASIS,ABDOMEN DISTENTION,NaN,1.000000,1.000000,3,0.001000
5,CHOLEDOCHOLITHIASIS,ABDOMEN GUARDING INVOLUNTARY LOCALIZED,NaN,1.000000,1.000000,3,0.001000
6,CHOLEDOCHOLITHIASIS,ABDOMEN MASS <ES> PRESENT <NON HEPATOSPLENIC>,NaN,1.000000,1.000000,3,0.001000
7,CHOLEDOCHOLITHIASIS,ABDOMEN MASS EPIGASTRIUM,NaN,1.000000,1.000000,3,0.001000
8,CHOLEDOCHOLITHIASIS,ABDOMEN MASS LEFT UPPER QUADRANT <NON SPLENIC>,NaN,1.000000,1.000000,3,0.001000
9,CHOLEDOCHOLITHIASIS,ABDOMEN MASS RIGHT LOWER QUADRANT,NaN,1.000000,1.000000,3,0.001000


In [62]:
df_test.to_csv('df_bayes.csv')